In [1]:
import torch
import sys
import os
os.environ["K_DIFFUSION_USE_COMPILE"] = "0"

import k_diffusion as K
import torch._C._onnx as _C_onnx
from torchinfo import summary
import cv2
import numpy as np
import torchvision

import pynvml
# import onnxruntime as ort
import os
import torch.nn as nn
from PIL import Image

/home/ubuntu/tdist-flat/tdist-regular/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_float32_matmul_precision('high')

In [3]:
config = "/home/ubuntu/tdist-flat/configs/hdit_shifted_window.json"
config = K.config.load_config(config)

model = K.config.make_model(config).cuda()

dct = torch.load("/home/ubuntu/tdist-flat/pix2pixHD/checkpoints/og-reg-4/epoch_18_netG.pth")
dct = {key.replace('model.', ''): value for key, value in dct.items()}

# Find unexpected keys
model_keys = set(model.state_dict().keys())
unexpected_keys = {key: value for key, value in dct.items() if key not in model_keys}

# Print or log the unexpected keys
print("Unexpected keys:", unexpected_keys.keys())

# Optionally, you can remove them before loading the state_dict
filtered_dct = {key: value for key, value in dct.items() if key in model_keys}

# Load the filtered state_dict
model.load_state_dict(filtered_dct, strict=False)

final_activation_function = nn.Tanh()
print(sum([p.numel() for p in model.parameters()]) / 1e6)
model.eval()

img = torch.load("/home/ubuntu/label.pt")[0].cuda().unsqueeze(0)

cst = torch.ones((img.shape[0])).cuda() * 500
with torch.no_grad(): out = final_activation_function(model(img, cst)[0])

out = out / 2. + .5
# raise ValueError(out.shape)
out = out.squeeze(0).transpose(0, 1).transpose(1, 2)
out = Image.fromarray((out.data.cpu().numpy() * 255.).astype(np.uint8))
out.save('out_shifted.png')

/tmp/ipykernel_2350966/3516763265.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dct = torch.load("/home/ubuntu/tdist-flat/pix2pixHD/checkpoints/og-reg-4/epoch_18_netG.

Unexpected keys: dict_keys(['alpha1', 'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha_sum', 'projector1.weight', 'projector2.weight', 'projector3.weight', 'projector4.weight', 'projector5.weight'])
85.087492


/tmp/ipykernel_2350966/3516763265.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img = torch.load("/home/ubuntu/label.pt")[0].cuda().unsqueeze(0)


In [4]:
class MyModel(torch.nn.Module):
    def __init__(self, model, device):
        super(MyModel, self).__init__()
        # Example: simple linear layer
        self.cst = torch.ones((1)).to(device) *500
        self.model = model

    def forward(self, img):
        x  = self.model(img,self.cst)
        return x

In [5]:
model.eval()

pmodel = MyModel(model, 'cuda')
pmodel = pmodel
of = '/home/ubuntu/og-ps4.onnx'
# input = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cuda()
# input = input.permute(0,2,3,1)
input = torch.rand(1,7,1024,1024).cpu()
torch.onnx.export(pmodel.cuda(), (input.cuda()), of,do_constant_folding=True, export_params=True,input_names = ['input'],output_names = ['output'], verbose=False)

/home/ubuntu/tdist-flat/kdiff-older/k_diffusion/models/axial_rope.py:54: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  y_min = torch.max(torch.tensor(-1), -1 / ar_adj)
/home/ubuntu/tdist-flat/kdiff-older/k_diffusion/models/axial_rope.py:55: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  y_max = torch.min(torch.tensor(1), 1 / ar_adj)
/home/ubuntu/tdist-flat/kdiff-older/k_diffusion/models/axial_rope.py:57: TracerWarning: torch.tensor results are registered as constants in the trac

TypeError: z_(): incompatible function arguments. The following argument types are supported:
    1. (self: torch._C.Node, arg0: str, arg1: torch.Tensor) -> torch._C.Node

Invoked with: %976 : Tensor = onnx::Constant(), scope: __main__.MyModel::/k_diffusion.models.image_transformer_v2.ImageTransformerDenoiserModelV2::model/k_diffusion.models.image_transformer_v2.Level::down_levels.0/k_diffusion.models.image_transformer_v2.ShiftedWindowTransformerLayer::down_levels.0.0/k_diffusion.models.image_transformer_v2.ShiftedWindowSelfAttentionBlock::self_attn
, 'value', 1.0 
(Occurred when translating scaled_dot_product_attention).

In [ ]:
import onnx
from onnx import shape_inference

# Load the ONNX model
model_path = 'channels-last.onnx'
model = onnx.load(model_path)

# Perform shape inference
inferred_model = shape_inference.infer_shapes(model)

# Save the inferred model (optional)
onnx.save(inferred_model, 'channels-last-inferred.onnx')


FileNotFoundError: [Errno 2] No such file or directory: 'channels-last.onnx'

In [ ]:
import onnxruntime as ort
import torch
import numpy as np

In [ ]:
print(sum([p.numel() for p in pmodel.parameters()]) / 1e6)

85.0793


In [ ]:
input = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cuda()
input = input.permute(0,2,3,1)

providers = [
    ('CPUExecutionProvider', {
        'device_id': 0, # The device ID
        # 'trt_max_workspace_size': 4e9, # Maximum workspace size for TensorRT engine (1e9 ≈ 1GB)
        # 'trt_engine_cache_enable': False, # Enable TensorRT engine caching
        # # 'trt_engine_cache_path': str(trt_cache_dir), # Path for TensorRT engine, profile files, and int8 calibration table
        # 'trt_int8_enable': True, # Enable int8 mode in TensorRT
    })
]

# providers = 
#     ('CUDAExecutionProvider')]

sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
sess_options.enable_mem_pattern = False
sess_options.use_deterministic_compute = True
sess_options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
sess_options.enable_cpu_mem_arena = False



session = ort.InferenceSession("channels-last-inferred.onnx",sess_options,providers=providers)

# Run inference with ONNX Runtime
input_data = input.cpu().numpy().astype(np.float32)
ort_inputs = {session.get_inputs()[0].name: input_data}
# start = torch.cuda.Event(enable_timing=True)
# end = torch.cuda.Event(enable_timing=True)

    # start.record()
onnx_output = session.run(None, ort_inputs)
    # end.record()
    # print(start.elapsed_time(end))
# Compare the outputs

# outim = np.transpose(out[0].data.cpu().numpy(), (1,2,0))
outim = out[0].data.cpu().numpy()
out = Image.fromarray((outim * 255.).astype(np.uint8))
out.save('out_shifted1.png')

/tmp/ipykernel_361958/3462181165.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  input = torch.load("/home/ubuntu/transformer-distillation/tensor_14.pt").unsqueeze(0).cu

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/transformer-distillation/tensor_14.pt'

In [ ]:
import onnx2keras
from onnx2keras import onnx_to_keras
import keras
import onnx


In [ ]:
out = torch.tanh(torch.tensor(onnx_output[0]))
out = out / 2. + .5


In [ ]:
out = Image.fromarray((outim * 255.).astype(np.uint8))
out.save('out_shifted1.png')